In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U transformers datasets scipy scikit-learn 
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.1 MB/s eta 0:00:00
Looking in index

## 문장 분류 모델 학습

노트북을 실행하는데 필요한 라이브러리들을 모두 임포트합니다.

In [ ]:
import random
import logging
from IPython.display import display, HTML


import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence,Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizerFast, AlbertForSequenceClassification

[링크 텍스트](https://)학습에 필요한 정보를 변수로 기록합니다.

본 노트북에서는 `klue-roberta-base` 모델을 활용하지만, https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

학습 태스크로는 `ynat`를, 배치 사이즈로는 64를 지정하겠습니다.


In [ ]:
model_checkpoint = 'bongsoo/albert-small-kor-v1'
batch_size = 64

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/텍스트분석2023/traindata/dataset.csv')

df["label"] = df["label"].astype(int)

In [ ]:


print(df)

from sklearn.model_selection import train_test_split
                                  
dataset_train, dataset_val = train_test_split(df, test_size=0.25, random_state=0)

dataset_train = dataset_train[['message','label']]
dataset_val = dataset_val[['message','label']]

tds = Dataset.from_pandas(dataset_train)
vds = Dataset.from_pandas(dataset_val)

datasets = DatasetDict()

datasets['train'] = tds
datasets['validation'] = vds

                                                message  label
0     the movie begins in the past where a young boy...      1
1     emerging from the human psyche and showing cha...      1
2     spurning her mother's insistence that she get ...      1
3     amitabh can't believe the board of directors a...      1
4     she , among others excentricities , talks to a...      1
...                                                 ...    ...
9994  a haunted , bountiful film that demands patien...      0
9995  the movie's gloomy atmosphere is fascinating ,...      0
9996  it aimlessly and unsuccessfully attempts to fu...      0
9997  an authentically vague , but ultimately purpos...      0
9998  visually imaginative , thematically instructiv...      0

[9999 rows x 2 columns]


각 예시 데이터는 아래와 같이 두 개의 문장과 두 문장의 추론 관계를 라벨로 지니고 있습니다.

In [ ]:
print(datasets["train"].shape)
print(datasets["validation"].shape)

(7499, 3)
(2500, 3)


In [ ]:
datasets["train"]

Dataset({
    features: ['message', 'label', '__index_level_0__'],
    num_rows: 7499
})

데이터셋을 전반적으로 살펴보기 위한 시각화 함수를 다음과 같이 정의합니다.

In [ ]:
datasets = datasets.remove_columns(['__index_level_0__'])

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['message', 'label'],
        num_rows: 7499
    })
    validation: Dataset({
        features: ['message', 'label'],
        num_rows: 2500
    })
})

훈련 과정 중 모델의 성능을 파악하기 위한 메트릭을 설정합니다.

`datasets` 라이브러리에는 이미 구현된 메트릭을 사용할 수 있는 `load_metric` 함수가 있습니다.

그 중 f1 score 로 metric을 설정하였습니다. 

In [ ]:
#metric.inputs_description
metric = load_metric("f1")

<ipython-input-11-f6db7250f76c>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


`accuracy` 메트릭이 정상적으로 작동하는지 확인하기 위해, 랜덤한 예측 값과 라벨 값을 생성합니다.

이제 학습에 활용할 토크나이저를 로드해오도록 합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

로드된 토크나이저가 두 개 문장을 토큰화하는 방식을 파악하기 위해 두 문장을 입력 값으로 넣어줘보도록 합시다.

`input_ids`를 보시면 `cls_token`에 해당하는 2번 토큰이 가장 좌측에 붙게 되며, `sep_token`의 3번 토큰이 각각 중간과 가장 우측에 더해진 것을 확인할 수 있습니다.


In [ ]:
tokenizer

AlbertTokenizer(name_or_path='bongsoo/albert-small-kor-v1', vocab_size=30000, model_max_length=128, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

이제 *key* 도 확인이 되었으니, 데이터셋에서 각 예제들을 뽑아와 토큰화 할 수 있는 함수를 아래와 같이 정의해줍니다.

해당 함수는 모델을 훈련하기 앞서 데이터셋을 미리 토큰화 시켜놓는 작업을 위한 콜백 함수로 사용되게 됩니다.

인자로 넣어주는 `truncation`는 모델이 입력 받을 수 있는 최대 길이 이상의 토큰 시퀀스가 들어오게 될 경우, 최대 길이 기준으로 시퀀스를 자르라는 의미를 지닙니다.

( \* `return_token_type_ids`는 토크나이저가 `token_type_ids`를 반환하도록 할 것인지를 결정하는 인자입니다. `transformers==4.7.0` 기준으로 `token_type_ids`가 기본적으로 반환되므로 `token_type_ids` 자체를 사용하지 않는 `RoBERTa` 모델을 활용하기 위해 해당 인자를 `False`로 설정해주도록 합니다.)

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples['message'],
        truncation=True,
        return_token_type_ids=False,
    )

앞서 정의한 `process_function`은 여러 개의 예제 데이터를 받을 수도 있습니다.

이제 정의된 전처리 함수를 활용해 데이터셋을 미리 토큰화시키는 작업을 수행합니다.

`datasets` 라이브러리를 통해 얻어진 `DatasetDict` 객체는 `map()` 함수를 지원하므로, 정의된 전처리 함수를 데이터셋 토큰화를 위한 콜백 함수로 `map()` 함수 인자로 넘겨주면 됩니다.

보다 자세한 내용은 [문서](https://huggingface.co/docs/datasets/processing.html#processing-data-with-map)를 참조해주시면 됩니다.

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/7499 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['message', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7499
    })
    validation: Dataset({
        features: ['message', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2500
    })
})

학습을 위한 모델을 로드할 차례입니다.

앞서 살펴본 바와 같이 **KLUE TC**에는 총 7개의 클래스가 존재하므로, 7개의 클래스를 예측할 수 있는 *SequenceClassification* 구조로 모델을 로드하도록 합니다.

In [ ]:
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bongsoo/albert-small-kor-v1 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at bongsoo

모델을 로드할 때 발생하는 경고 문구는 두 가지 의미를 지닙니다.

1. *Masked Language Modeling* 을 위해 존재했던 `lm_head`가 현재는 사용되지 않고 있음을 의미합니다.
2. 문장 분류를 위한 `classifier` 레이어를 백본 모델 뒤에 이어 붙였으나 아직 훈련이 되지 않았으므로, 학습을 수행해야 함을 의미합니다.

마지막으로 앞서 정의한 메트릭을 모델 예측 결과에 적용하기 위한 함수를 정의합니다.

입력으로 들어오는 `eval_pred`는 [*EvalPrediction*](https://huggingface.co/transformers/internal/trainer_utils.html#transformers.EvalPrediction) 객체이며, 모델의 클래스 별 예측 값과 정답 값을 지닙니다.

클래스 별 예측 중 가장 높은 라벨을 `argmax()`를 통해 뽑아낸 후, 정답 라벨과 비교를 하게 됩니다.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average = 'macro')

In [ ]:
!pip install --upgrade accelerate
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.28.0-py3-none-any.whl (7.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [ ]:
metric_name = "f1"

args = TrainingArguments(
    "test-tc",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

이제 로드한 모델, 인자 관리 클래스, 데이터셋 등을 *Trainer* 클래스를 초기화에 넘겨주도록 합니다.

(TIP: Q: 이미 `encoded_datasets`을 만드는 과정에 토큰화가 이루어졌는데 토크나이저를 굳이 넘겨주는 이유가 무엇인가요?,<br>A: 토큰화는 이루어졌지만 학습 과정 시, 데이터를 배치 단위로 넘겨주는 과정에서 배치에 포함된 가장 긴 시퀀스 기준으로 `truncation`을 수행하고 최대 길이 시퀀스 보다 짧은 시퀀스들은 그 길이만큼 `padding`을 수행해주기 위함입니다.)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  super().__init__(params, defaults)


Epoch,Training Loss,Validation Loss,F1
1,No log,0.332195,0.853990
2,No log,0.283180,0.882751
3,No log,0.306087,0.873365
4,No log,0.316461,0.877848
5,0.245400,0.321669,0.889954
6,0.245400,0.358225,0.893192
7,0.245400,0.412260,0.889943


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   )                                                                                 │
│   1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│ ❱ 1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│   1666 │   │   │   ignore_keys_for_eval=ignore_keys_for_eval,                                    │
│   1667 │   │   )                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1940 in _inner_training_loop     │
│                                                                                                  │
│   1937 │   │   │   │   │   tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_  │
│   1938 │   │   │   │   else:                                                                     │
│   1939 │   │   │   │   │   tr_loss += tr_loss_step                                               │
│ ❱ 1940 │   │   │   │                                                                             │
│   1941 │   │   │   │   self.current_flos += float(self.floating_point_ops(inputs))               │
│   1942 │   │   │   │                                                                             │
│   1943 │   │   │   │   # Optimizer step for deepspeed must be called on every step regardless o  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2753 in training_step            │
│                                                                                                  │
│   2750 │   │                                                                                     │
│   2751 │   │   return (loss, outputs) if return_outputs else loss                                │
│   2752 │                                                                                         │
│ ❱ 2753 │   def is_local_process_zero(self) -> bool:                                              │
│   2754 │   │   """                                                                               │
│   2755 │   │   Whether or not this process is the local (e.g., on one machine if training in a   │
│   2756 │   │   machines) main process.                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_gra

*Trainer* 는 학습을 마치게 되면, `load_best_model_at_end=True` 인자에 따라 메트릭 기준 가장 좋은 성능을 보였던 체크포인트를 로드하게 됩니다.

본 노트북에서는 마지막 에폭 때 가장 좋은 성능을 얻었기에 `evaluate`를 수행해도 같은 결과가 나오겠습니다.

In [ ]:
# trainer.evaluate()

In [ ]:
# import os
# os.system("mv /content/test-tc/checkpoint-590 /content/drive/MyDrive/텍스트분석2023/model/")

0

## 추론


In [ ]:
# !pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0
    Uninstalling transformers-4.28.0:
      Successfully uninstalled transformers-4.28.0


In [ ]:
# pip install numpy==1.24.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.


In [3]:
pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━

In [4]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="/content/drive/MyDrive/텍스트분석2023/model/checkpoint-590",
    return_all_scores=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


0 (subjective), 1(objective)

In [ ]:
df1['eng'].isnull().unique()

array([False])

In [5]:
import pandas as pd
df1 = pd.read_excel('/content/drive/MyDrive/텍스트분석2023/merge_data/Evinue_translate.xlsx')

max_input_length = 512  # 모델의 기대 입력 크기
df1_s = []
df1_o = []
for data in df1['eng']:
  if len(data) > max_input_length:
    data = data[:max_input_length]  # 입력 데이터를 앞에서부터 max_input_length만큼 잘라냄
  tmp = classifier(data)
  s = tmp[0][0]['score']
  o = tmp[0][1]['score']
  df1_s.append(s)
  df1_o.append(o)

Token indices sequence length is longer than the specified maximum sequence length for this model (181 > 128). Running this sequence through the model will result in indexing errors


In [6]:
df1['subjective'] = df1_s
df1['objective'] = df1_o
print(df1)

                                                  리뷰 내용  \
0     눈시림 있는 선크림이 많은데 이건 전혀 눈시림이 없어요 바르면 얼굴이 번쩍거릴 정도...   
1     한창 쓰다가 다른제품으로 갈아탔다가 이번에 세일하길래 다시 구매했어요 그런데 다른제...   
2                       남편 주려고 주문해봤어요바르고 눈 비비니 눈이 따끔거려요   
3     헤라 믿쓰화장품 중 하나지요어머니께 공수 받은 애터미 선크림 쓰다가 이것도 가성비갑...   
4            냄새는 그닥번들거림 약간있는편톤이 밝아지는건 없음무난한편 눈 따가움 없어좋음   
...                                                 ...   
1045  와 인생선크림 찾기가 이렇게 힘든가요좋다는거 다써봤지만 왜 헤라꺼는 안샀었는지 후회...   
1046                             밀착감이 너무 좋고 자극이 없어서 좋아요   
1047       잘발리고 가볍고 좋아요 텁텁하지도 않고 많이 건조하지도 않고 적당하고 딱 좋아요   
1048  어떤것들은 기초후에 바르면 밀려서 닦아내곤 하는데 이건 발림성 도좋고 촉촉하니 랑큼...   
1049  메이크업 쪽은 잘 몰라서 10년 째 그냥 헤라 제품 사용 합니다톤 업이라 해서 살짝...   

                                                    eng  subjective  objective  
0     There are a lot of sun creams that hurt my eye...    0.982125   0.017875  
1     I used it for a while and changed to another p...    0.988025   0.011975  
2     I ordered it for my husbandMy eyes sting w

In [7]:
df1.to_excel('/content/drive/MyDrive/텍스트분석2023/merge_data/Evinue_subobj.xlsx')

In [12]:
df2 = pd.read_excel('/content/drive/MyDrive/텍스트분석2023/merge_data/NoEvinue_translate.xlsx')
df2_s = []
df2_o = []
for data in df2['eng']:
  if len(data) > max_input_length:
    data = data[:max_input_length]  # 입력 데이터를 앞에서부터 max_input_length만큼 잘라냄
  tmp = classifier(data)
  s = tmp[0][0]['score']
  o = tmp[0][1]['score']
  df2_s.append(s)
  df2_o.append(o)

In [9]:
df2

,Unnamed: 0,리뷰 내용
0,0,주변에서 추천을 너무 많이 하셔서 한번 사봤는데 대만족입니다선크림 특히나 톤업 선크...
1,1,마스크도 벗고 야외활동도 가능해진 요즘 화장은 필수죠 그래서 달바를 선택했어요 이제...
2,2,엄청 더운 요즘 정말이지 가만히 있어도 땀이 줄줄줄아이 학교 왔다갔다 둘째 얼집왔다...
3,3,날씨가 풀리면서낮에 아기데리고 산책도 자주하고요즘 햇볕도 많이 뜨거워졌네요 화장도 ...
4,4,2개를 번갈아 가면서 사용중인데 워터풀 선크림은 만족스럽습니다 보습력이 좋아요 근데...
...,...,...
1045,1045,다이브인세럼을 여러개 쓰고 좋아서 이것도 사봤어요 원래 쓰던것도 좋았는데 토리든 성...
1046,1046,바르면 무거운느낌은 아니고 가볍게 데일리로 바를만 합니다
1047,1047,배송속도 하루 지방도시 포장상태 쿠팡 비닐포장사용자 피부타입 민감 수부지 붉은기 색...
1048,1048,성인여드름으로 sns에 뜨는 여드름에 좋다고 하면 혹해서 샀었는데 저는 어느 것도 ...


In [18]:
new_df2 = df2.drop(labels='Unnamed: 0',axis=1)

In [19]:
new_df2

,리뷰 내용,eng,subjective,objective
0,주변에서 추천을 너무 많이 하셔서 한번 사봤는데 대만족입니다선크림 특히나 톤업 선크...,A lot of people recommended it so I bought it ...,0.977986,0.022014
1,마스크도 벗고 야외활동도 가능해진 요즘 화장은 필수죠 그래서 달바를 선택했어요 이제...,Now that you can take off your mask and do out...,0.430656,0.569344
2,엄청 더운 요즘 정말이지 가만히 있어도 땀이 줄줄줄아이 학교 왔다갔다 둘째 얼집왔다...,It s really hot these days I went to school wh...,0.638594,0.361406
3,날씨가 풀리면서낮에 아기데리고 산책도 자주하고요즘 햇볕도 많이 뜨거워졌네요 화장도 ...,As the weather gets warmer I take babies for w...,0.806109,0.193891
4,2개를 번갈아 가면서 사용중인데 워터풀 선크림은 만족스럽습니다 보습력이 좋아요 근데...,I m using two alternately but I m satisfied wi...,0.997510,0.002490
...,...,...,...,...
1045,다이브인세럼을 여러개 쓰고 좋아서 이것도 사봤어요 원래 쓰던것도 좋았는데 토리든 성...,I used several dive in serum and I bought this...,0.979820,0.020180
1046,바르면 무거운느낌은 아니고 가볍게 데일리로 바를만 합니다,It doesn t feel heavy but it s easy to apply o...,0.992903,0.007097
1047,배송속도 하루 지방도시 포장상태 쿠팡 비닐포장사용자 피부타입 민감 수부지 붉은기 색...,Delivery speed One day local city Packaging st...,0.411176,0.588824
1048,성인여드름으로 sns에 뜨는 여드름에 좋다고 하면 혹해서 샀었는데 저는 어느 것도 ...,I bought some pimples when I was good in adult...,0.997592,0.002408


In [13]:
df2['subjective'] = df2_s
df2['objective'] = df2_o
print(df2)

      Unnamed: 0                                              리뷰 내용  \
0              0  주변에서 추천을 너무 많이 하셔서 한번 사봤는데 대만족입니다선크림 특히나 톤업 선크...   
1              1  마스크도 벗고 야외활동도 가능해진 요즘 화장은 필수죠 그래서 달바를 선택했어요 이제...   
2              2  엄청 더운 요즘 정말이지 가만히 있어도 땀이 줄줄줄아이 학교 왔다갔다 둘째 얼집왔다...   
3              3  날씨가 풀리면서낮에 아기데리고 산책도 자주하고요즘 햇볕도 많이 뜨거워졌네요 화장도 ...   
4              4  2개를 번갈아 가면서 사용중인데 워터풀 선크림은 만족스럽습니다 보습력이 좋아요 근데...   
...          ...                                                ...   
1045        1045  다이브인세럼을 여러개 쓰고 좋아서 이것도 사봤어요 원래 쓰던것도 좋았는데 토리든 성...   
1046        1046                    바르면 무거운느낌은 아니고 가볍게 데일리로 바를만 합니다   
1047        1047  배송속도 하루 지방도시 포장상태 쿠팡 비닐포장사용자 피부타입 민감 수부지 붉은기 색...   
1048        1048  성인여드름으로 sns에 뜨는 여드름에 좋다고 하면 혹해서 샀었는데 저는 어느 것도 ...   
1049        1049  쿠팡 체험단 후기입니다 저는 민감성 피부이며 홍조피부에요 아무리 좋다는 제품발라도 ...   

                                                    eng  subjective  objective  
0     A lot of people recommended it so I bought it ...    0.97798

In [20]:
new_df2.to_excel('/content/drive/MyDrive/텍스트분석2023/merge_data/NoEvinue_subobj.xlsx')

In [ ]:
# classifier(outStr2)

[[{'label': 'LABEL_0', 'score': 0.9973418116569519},
  {'label': 'LABEL_1', 'score': 0.002658255398273468}]]